In [ ]:
## For cleaning GA Data. Remember that the goal here is to: 
## (1) Turn our policy results into something the machine algorithm can easily read
## (2) Sort the individual motions into their appropriate categories. This will help us later discuss the implications of our machine's predictions.
## (3) Delete data used for stress testing - such as the meeting number.  

### IMPORTANT: You must run the code in the order presented. 

import numpy as np 
import pandas as pd

import matplotlib
import matplotlib.pyplot as pp

import csv

In [ ]:
#Reads and displays the Data file
dfSC = pd.read_csv('SC_Queries.csv')
dfSC

,Unnamed: 0,meetings,dates,May I take it that the assembly wishes to take note of those items that remain open for consideration,The agenda was adopted,appoint a committee,I have been authorized to make the following statement on behalf of the committee,May I take it that the draft report as corrected is adopted by the council,point of order,amendment,A paragraph-by-paragraph vote on the draft resolution,refer the matter,postpone discussion to,postpone discussion indefinitely,The meeting was suspended,the meeting is adjourned,To withdraw the draft resolution,"In conformity with the usual practice, I propose, with consent of the council, to invite those representatives to participate in discussion",It is my understanding that the security council is ready to vote on the draft resolution before it,peacekeeping,peacemaking,sanctions,The draft resolution has been adopted,The draft resolution has not been adopted
0,0,pdf?symbol=en%2FS%2FPV.7982,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,1,0,0
1,1,pdf?symbol=en%2FS%2FPV.6900,2013,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,12,0,0
2,2,pdf?symbol=en%2FS%2FPV.8428,2018,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,41,0,0
3,3,pdf?symbol=en%2FS%2FPV.7797,2016,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,pdf?symbol=en%2FS%2FPV.3532,1995,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,5082,pdf?symbol=en%2FS%2FPV.7865,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0
5083,5083,pdf?symbol=en%2FS%2FPV.7980,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5,0,0
5084,5084,pdf?symbol=en%2FS%2FPV.4571,2002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5085,5085,pdf?symbol=en%2FS%2FPV.8322,2018,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0


In [ ]:
###First we need to subtract 'Not been adopted' from 'has been adopted' 
#Note: This was used for confirming a stress test.

dfSC['The draft resolution has been adopted'] = dfSC['The draft resolution has been adopted'] - dfSC['The draft resolution has not been adopted']

dfSC.head(20)

,Unnamed: 0,meetings,dates,May I take it that the assembly wishes to take note of those items that remain open for consideration,The agenda was adopted,appoint a committee,I have been authorized to make the following statement on behalf of the committee,May I take it that the draft report as corrected is adopted by the council,point of order,amendment,A paragraph-by-paragraph vote on the draft resolution,refer the matter,postpone discussion to,postpone discussion indefinitely,The meeting was suspended,the meeting is adjourned,To withdraw the draft resolution,"In conformity with the usual practice, I propose, with consent of the council, to invite those representatives to participate in discussion",It is my understanding that the security council is ready to vote on the draft resolution before it,peacekeeping,peacemaking,sanctions,The draft resolution has been adopted,The draft resolution has not been adopted
0,0,pdf?symbol=en%2FS%2FPV.7982,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,1,0,0
1,1,pdf?symbol=en%2FS%2FPV.6900,2013,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,12,0,0
2,2,pdf?symbol=en%2FS%2FPV.8428,2018,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,41,0,0
3,3,pdf?symbol=en%2FS%2FPV.7797,2016,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,pdf?symbol=en%2FS%2FPV.3532,1995,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,pdf?symbol=en%2FS%2FPV.8127,2017,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8,0,38,0,0
6,6,pdf?symbol=en%2FS%2FPV.4272,2001,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,61,2,1,0,0
7,7,pdf?symbol=en%2FS%2FPV.5304,2005,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
8,8,pdf?symbol=en%2FS%2FPV.6521,2011,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,9,pdf?symbol=en%2FS%2FPV.6979,2013,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
# Next, we're going to sort the motions by their categories. 

dfSC['Class M'] = dfSC['The agenda was adopted'] 
+ dfSC['appoint a committee'] 
+ dfSC['I have been authorized to make the following statement on behalf of the committee'] 
+ dfSC['May I take it that the draft report as corrected is adopted by the council']
+ dfSC['In conformity with the usual practice, I propose, with consent of the council, to invite those representatives to participate in discussion']

dfSC['Class S'] = dfSC['amendment'] 
+ dfSC['refer the matter'] 
+ dfSC['postpone discussion to']
+ dfSC['postpone discussion indefinitely']
+ dfSC['It is my understanding that the security council is ready to vote on the draft resolution before it']

dfSC['Class I'] = dfSC['point of order']
+ dfSC['A paragraph-by-paragraph vote on the draft resolution']
+ dfSC['To withdraw the draft resolution']

dfSC['Class P'] = dfSC['The meeting was suspended'] + dfSC['the meeting is adjourned']

dfSC['Policy Passed'] = dfSC['The draft resolution has been adopted']

dfSC['Conflict Indicator'] = dfSC['sanctions'] + dfSC['peacekeeping'] + dfSC['peacemaking']

dfSC

,Unnamed: 0,meetings,dates,May I take it that the assembly wishes to take note of those items that remain open for consideration,The agenda was adopted,appoint a committee,I have been authorized to make the following statement on behalf of the committee,May I take it that the draft report as corrected is adopted by the council,point of order,amendment,A paragraph-by-paragraph vote on the draft resolution,refer the matter,postpone discussion to,postpone discussion indefinitely,The meeting was suspended,the meeting is adjourned,To withdraw the draft resolution,"In conformity with the usual practice, I propose, with consent of the council, to invite those representatives to participate in discussion",It is my understanding that the security council is ready to vote on the draft resolution before it,peacekeeping,peacemaking,sanctions,The draft resolution has been adopted,The draft resolution has not been adopted,Class M,Class S,Class I,Class P,Policy Passed,Conflict Indicator
0,0,pdf?symbol=en%2FS%2FPV.7982,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,1,0,0,1,0,0,0,0,6
1,1,pdf?symbol=en%2FS%2FPV.6900,2013,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,12,0,0,1,0,0,1,0,13
2,2,pdf?symbol=en%2FS%2FPV.8428,2018,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,41,0,0,1,0,0,0,0,44
3,3,pdf?symbol=en%2FS%2FPV.7797,2016,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
4,4,pdf?symbol=en%2FS%2FPV.3532,1995,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,5082,pdf?symbol=en%2FS%2FPV.7865,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,0,0,1,0,0,0,0,25
5083,5083,pdf?symbol=en%2FS%2FPV.7980,2017,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,5,0,0,1,0,0,0,0,6
5084,5084,pdf?symbol=en%2FS%2FPV.4571,2002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5085,5085,pdf?symbol=en%2FS%2FPV.8322,2018,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,1,0,0,0,0,22


In [ ]:
#Delete columns of queries not needed and now summarized. 
#Deleted columns include: motion queries, meeting records for stress testing

dfSC.drop(['Unnamed: 0', 
           'May I take it that the assembly wishes to take note of those items that remain open for consideration',
           'The agenda was adopted',
           'appoint a committee',
           'I have been authorized to make the following statement on behalf of the committee',
           'May I take it that the draft report as corrected is adopted by the council',
           'point of order',
           'amendment',
           'A paragraph-by-paragraph vote on the draft resolution',
           'refer the matter',
           'postpone discussion to',
           'postpone discussion indefinitely',
           'The meeting was suspended',
           'the meeting is adjourned',
           'To withdraw the draft resolution',
           'In conformity with the usual practice, I propose, with consent of the council, to invite those representatives to participate in discussion',
           'It is my understanding that the security council is ready to vote on the draft resolution before it',    
           'The draft resolution has not been adopted',
           'The draft resolution has been adopted',
           'sanctions',
           'peacekeeping',
           'peacemaking',
          ], axis = 1, inplace= True)

dfSC

,meetings,dates,Class M,Class S,Class I,Class P,Policy Passed,Conflict Indicator
0,pdf?symbol=en%2FS%2FPV.7982,2017,1,0,0,0,0,6
1,pdf?symbol=en%2FS%2FPV.6900,2013,1,0,0,1,0,13
2,pdf?symbol=en%2FS%2FPV.8428,2018,1,0,0,0,0,44
3,pdf?symbol=en%2FS%2FPV.7797,2016,1,0,1,0,0,0
4,pdf?symbol=en%2FS%2FPV.3532,1995,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
5082,pdf?symbol=en%2FS%2FPV.7865,2017,1,0,0,0,0,25
5083,pdf?symbol=en%2FS%2FPV.7980,2017,1,0,0,0,0,6
5084,pdf?symbol=en%2FS%2FPV.4571,2002,0,0,0,0,0,0
5085,pdf?symbol=en%2FS%2FPV.8322,2018,1,0,0,0,0,22


In [ ]:
#Transform the Policy Passed Label and Conflict Indicators into a binary for classification
# Do not keep as a frequency - query stress tested for meeting passing policy, not the measurement of that policy (some meetings have 50+ passed policies on technical issues)
# This is necessary to properly run Sigmoid activation function. If you want to keep policy counts, you will need to change 
#the label activation function

a = np.array(dfSC['Policy Passed'].values.tolist())
print(a)
dfSC['Policy Passed'] = np.where(a > 1, 1, a).tolist()

b = np.array(dfSC['Conflict Indicator'].values.tolist())
print(b)
dfSC['Conflict Indicator'] = np.where(b > 1, 1, b).tolist()

dfSC.head(20)

[0 0 0 ... 0 0 0]
[ 6 13 44 ...  0 22  0]


,meetings,dates,Class M,Class S,Class I,Class P,Policy Passed,Conflict Indicator
0,pdf?symbol=en%2FS%2FPV.7982,2017,1,0,0,0,0,1
1,pdf?symbol=en%2FS%2FPV.6900,2013,1,0,0,1,0,1
2,pdf?symbol=en%2FS%2FPV.8428,2018,1,0,0,0,0,1
3,pdf?symbol=en%2FS%2FPV.7797,2016,1,0,1,0,0,0
4,pdf?symbol=en%2FS%2FPV.3532,1995,1,0,0,0,0,0
5,pdf?symbol=en%2FS%2FPV.8127,2017,1,1,0,0,0,1
6,pdf?symbol=en%2FS%2FPV.4272,2001,1,0,0,1,0,1
7,pdf?symbol=en%2FS%2FPV.5304,2005,1,0,0,0,1,0
8,pdf?symbol=en%2FS%2FPV.6521,2011,1,0,0,0,0,0
9,pdf?symbol=en%2FS%2FPV.6979,2013,1,0,0,0,0,1


In [ ]:
LIWC_SC = pd.read_csv('SC_LIWC_Query.csv')

LIWC_SC

,Filename,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,...,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,pdf_symbol=en_2FS_2FPV.3326.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pdf_symbol=en_2FS_2FPV.3327.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pdf_symbol=en_2FS_2FPV.3328.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pdf_symbol=en_2FS_2FPV.3329.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pdf_symbol=en_2FS_2FPV.3330.pdf,1.0,940.0,99.00,81.71,9.67,69.86,28.48,34.04,72.66,43.72,6.17,3.62,1.60,0.32,0.53,0.53,0.64,2.55,12.45,17.34,3.94,1.49,3.51,0.11,6.81,4.57,1.70,0.43,3.30,0.43,5.64,3.83,1.49,0.00,0.21,0.85,10.00,0.00,0.32,...,0.11,0.00,0.32,8.19,1.38,1.60,4.04,0.74,1.06,1.49,3.83,0.64,11.06,0.96,7.98,2.13,8.30,0.53,0.21,0.00,0.11,0.11,0.43,0.00,0.11,0.11,0.21,0.0,22.34,5.64,3.30,0.74,0.00,0.00,0.0,0.96,0.85,0.32,1.17,9.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5375,pdf_symbol=en_2FS_2FPV.8741.pdf,1.0,15069.0,97.23,78.83,20.24,92.55,22.83,29.28,79.23,46.01,7.39,3.84,0.87,1.82,0.07,0.34,0.74,3.54,10.16,16.62,5.18,2.59,5.24,0.44,8.66,3.93,1.69,0.67,2.38,1.21,7.47,5.80,1.61,0.27,0.46,0.17,8.57,0.11,0.18,...,0.28,0.13,0.13,10.68,3.80,2.35,3.52,0.88,1.17,1.63,6.48,1.00,14.59,1.23,9.46,4.04,5.02,0.56,0.19,0.48,0.17,0.10,0.35,0.01,0.15,0.06,0.13,0.0,15.09,8.08,3.93,0.05,0.07,0.01,0.0,1.00,0.07,0.52,0.64,0.72
5376,pdf_symbol=en_2FS_2FPV.8742.pdf,1.0,5912.0,98.34,82.22,14.74,97.72,23.46,30.43,76.17,45.57,6.72,3.26,0.41,2.05,0.03,0.27,0.51,3.45,10.74,17.24,4.50,1.89,5.33,0.44,7.92,3.55,1.83,0.32,2.52,1.52,7.61,6.44,1.13,0.25,0.30,0.07,9.00,0.03,0.34,...,0.10,0.00,0.12,10.03,4.21,2.10,2.71,1.20,0.96,1.37,5.56,1.47,12.74,1.34,7.32,4.01,4.53,0.68,0.17,0.22,0.34,0.08,0.52,0.03,0.30,0.03,0.15,0.0,20.69,12.97,4.03,0.07,0.02,0.00,0.0,1.32,0.00,0.56,1.05,0.68
5377,pdf_symbol=en_2FS_2FPV.8743.pdf,1.0,23821.0,97.08,72.48,16.65,21.82,26.26,34.49,77.10,43.33,5.98,2.82,0.63,1.44,0.10,0.09,0.56,3.16,8.57,16.62,4.16,2.24,6.81,0.44,7.16,3.69,1.71,0.55,2.33,1.60,8.70,4.19,4.45,2.57,1.42,0.18,6.86,0.05,0.23,...,0.37,0.02,0.09,13.86,3.72,2.40,6.07,1.05,1.80,1.14,5.37,0.43,12.44,1.23,8.89,2.24,5.29,0.33,0.22,0.71,0.40,0.16,0.36,0.01,0.20

In [ ]:
#Merge with LIWC 

LIWC_SC = pd.read_csv('SC_LIWC_Query.csv')

LIWC_SC['Filename'] = LIWC_SC['Filename'].map(lambda x: str(x)[:-4]) ### EDIT
dfSC['meetings'] = dfSC['meetings'].map(lambda x:str(x.replace("%","_").replace("?","_")))

dfSC1 = dfSC.merge(LIWC_SC, left_on = "meetings", right_on = "Filename")

dfSC1["meetings"] = dfSC1["meetings"].map(lambda x: x if len(x) == 4 else x.split("/")[0])
dfSC1

,meetings,dates,Class M,Class S,Class I,Class P,Policy Passed,Conflict Indicator,Filename,Segment,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,...,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,pdf_symbol=en_2FS_2FPV.7982,2017,1,0,0,0,0,1,pdf_symbol=en_2FS_2FPV.7982,1.0,6647.0,98.36,71.97,22.23,70.50,26.48,31.31,77.43,45.63,6.38,2.77,0.63,1.17,0.08,0.20,0.69,3.61,11.37,16.70,4.90,2.17,4.96,0.50,8.02,4.65,1.58,0.62,2.63,1.26,5.31,...,0.29,0.00,0.12,11.00,2.98,1.93,4.57,0.80,1.49,1.76,5.61,0.87,14.55,1.61,9.19,3.81,5.60,0.51,0.18,0.26,0.06,0.12,0.30,0.0,0.15,0.02,0.14,0.0,20.43,12.20,4.23,0.09,0.03,0.00,0.0,1.35,0.14,0.39,1.11,0.89
1,pdf_symbol=en_2FS_2FPV.6900,2013,1,0,0,1,0,1,pdf_symbol=en_2FS_2FPV.6900,1.0,22867.0,97.08,70.99,19.29,11.75,23.75,36.34,79.70,44.04,6.72,2.93,0.57,1.60,0.09,0.18,0.49,3.79,9.06,16.54,4.67,2.19,5.94,0.51,7.15,4.05,1.95,0.62,1.83,1.53,9.59,...,0.29,0.00,0.13,15.28,3.48,2.91,7.30,1.26,1.72,1.13,5.65,0.49,12.63,1.20,8.76,2.61,5.58,0.28,0.17,0.59,0.30,0.13,0.20,0.0,0.08,0.03,0.09,0.0,14.47,6.64,4.83,0.06,0.05,0.01,0.0,1.49,0.04,0.42,0.66,0.27
2,pdf_symbol=en_2FS_2FPV.8428,2018,1,0,0,0,0,1,pdf_symbol=en_2FS_2FPV.8428,1.0,8394.0,98.40,76.05,13.66,91.35,25.99,31.43,78.15,46.06,7.15,3.47,1.18,1.11,0.01,0.24,0.93,3.68,10.88,17.61,4.46,2.10,5.04,0.43,8.35,3.63,1.80,0.55,3.65,1.97,5.53,...,0.14,0.02,0.08,10.96,3.76,3.06,2.84,1.02,1.20,1.85,5.68,0.79,11.25,1.19,6.80,3.48,7.56,0.43,0.08,0.51,0.04,0.01,0.38,0.0,0.19,0.07,0.12,0.0,17.68,10.03,4.21,0.11,0.11,0.04,0.0,0.67,0.02,0.41,1.44,0.66
3,pdf_symbol=en_2FS_2FPV.7797,2016,1,0,1,0,0,0,pdf_symbol=en_2FS_2FPV.7797,1.0,22044.0,96.04,72.47,20.26,37.49,23.75,26.91,79.97,47.55,7.94,3.80,1.14,1.40,0.08,0.28,0.89,4.14,10.35,16.03,5.99,2.79,5.30,0.73,10.07,4.19,2.02,0.94,2.79,1.73,6.81,...,0.71,0.00,0.42,10.22,2.94,1.83,4.10,0.90,1.31,2.40,6.76,0.82,13.40,1.24,8.35,3.85,4.13,0.76,0.14,0.65,0.20,0.44,0.39,0.0,0.26,0.04,0.08,0.0,14.28,6.91,4.38,0.09,0.15,0.05,0.0,1.07,0.10,0.64,0.37,0.54
4,pdf_symbol=en_2FS_2FPV.3532,1995,1,0,0,0,0,0,pdf_symbol=en_2FS_2FPV.3532,1.0,446.0,99.00,74.23,16.88,55.35,13.52,29.82,71.75,39.24,4.26,0.67,0.45,0.00,0.00,0.00,0.22,3.59,13.45,15.25,4.04,0.90,1.35,0.00,6.73,1.57,0.00,0.00,4.48,0.90,6.73,...,0.00,0.00,0.00,9.19,1.35,0.90,4.71,0.45,2.24,2.69,2.47,1.35,12.56,0.90,6.95,4.48,6.28,0.00,0.22,0.22,0.00,1.35,0.45,0.0,0.45,0.00,0.00,0.0,133.41,128.48,1.79,0.67,0.00,0.00,0.0,0.67,0.00,0.22,1.35,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,pdf_symbol=en_2FS_2FPV.7865,2017,1,0,0,0,0,1,pdf_symbol=en_2FS_2FPV.7865,1.0,15470.0,97.88,70.65,19.22,85.54,27.19,29.61,75.84,46.05,7.32,3.41,0.92,1.54,0.12,0.38,0.45,3.92,10.34,17.31,4.82,2.46,5.00,0.54,8.13,4.09,1.71,0.72,4.43,1.47,5.22,...,0.18,0.01,0.08,8.84,2.82,2.34,2.59,1.16,0.95,1.71,5.69,1.00,12.00,1.41,6.93,3.76,5.27,0.50,0.10,0.46,0.15,0.05,0.40,0.0,0.23,0.06,0.12,0.0,17.43,7.33,4.45,0.08,0.06,0.01,0.0,1.46,0.12,0.48,2.39,1.05
5075,pdf_symbol=en_2FS_2FPV.7980,2017,1,0,0,0,0,1,pdf_symbol=en_2FS_2FPV.7980,1.0,24988.0,97.96,74.86,19.68,68.73,24.62,33.85,77.60,45.02,7.01,3.43,0.73,1.85,0.03,0.35,0.46,3.58,10.00,17.09,4.26,2.04,5.66,0.42,7.58,4.35,1.93,0.62,2.35,1.46,7.88,...,0.30,0.00,0.14,12.04,3.68,2.68,4.47,0.92,1.27,1.45,5.48,0.94,13.84,1.26,9.16,3.43,5.37,0.39,0.19,0.44,0.29,0.22,0.39,0.0,0.19,0.04,0.16,0.0,14.39,6.38,4.37,0.05,0.06,0.04,0

In [ ]:
#Final Cleaning
# WARNING: change to dfGA1 if you have needed to combine separate files. 

#dfGA1.drop([''], axis = 1, inplace= True) #Specify any additional columns created to delete - usually 'unnamed'

dfSC1.drop(['meetings', 
           'Filename', 
           'Segment'], axis = 1, inplace= True)

dfSC1

,dates,Class M,Class S,Class I,Class P,Policy Passed,Conflict Indicator,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,...,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,2017,1,0,0,0,0,1,6647.0,98.36,71.97,22.23,70.50,26.48,31.31,77.43,45.63,6.38,2.77,0.63,1.17,0.08,0.20,0.69,3.61,11.37,16.70,4.90,2.17,4.96,0.50,8.02,4.65,1.58,0.62,2.63,1.26,5.31,3.84,1.46,0.20,...,0.29,0.00,0.12,11.00,2.98,1.93,4.57,0.80,1.49,1.76,5.61,0.87,14.55,1.61,9.19,3.81,5.60,0.51,0.18,0.26,0.06,0.12,0.30,0.0,0.15,0.02,0.14,0.0,20.43,12.20,4.23,0.09,0.03,0.00,0.0,1.35,0.14,0.39,1.11,0.89
1,2013,1,0,0,1,0,1,22867.0,97.08,70.99,19.29,11.75,23.75,36.34,79.70,44.04,6.72,2.93,0.57,1.60,0.09,0.18,0.49,3.79,9.06,16.54,4.67,2.19,5.94,0.51,7.15,4.05,1.95,0.62,1.83,1.53,9.59,4.23,5.31,3.56,...,0.29,0.00,0.13,15.28,3.48,2.91,7.30,1.26,1.72,1.13,5.65,0.49,12.63,1.20,8.76,2.61,5.58,0.28,0.17,0.59,0.30,0.13,0.20,0.0,0.08,0.03,0.09,0.0,14.47,6.64,4.83,0.06,0.05,0.01,0.0,1.49,0.04,0.42,0.66,0.27
2,2018,1,0,0,0,0,1,8394.0,98.40,76.05,13.66,91.35,25.99,31.43,78.15,46.06,7.15,3.47,1.18,1.11,0.01,0.24,0.93,3.68,10.88,17.61,4.46,2.10,5.04,0.43,8.35,3.63,1.80,0.55,3.65,1.97,5.53,4.74,0.71,0.15,...,0.14,0.02,0.08,10.96,3.76,3.06,2.84,1.02,1.20,1.85,5.68,0.79,11.25,1.19,6.80,3.48,7.56,0.43,0.08,0.51,0.04,0.01,0.38,0.0,0.19,0.07,0.12,0.0,17.68,10.03,4.21,0.11,0.11,0.04,0.0,0.67,0.02,0.41,1.44,0.66
3,2016,1,0,1,0,0,0,22044.0,96.04,72.47,20.26,37.49,23.75,26.91,79.97,47.55,7.94,3.80,1.14,1.40,0.08,0.28,0.89,4.14,10.35,16.03,5.99,2.79,5.30,0.73,10.07,4.19,2.02,0.94,2.79,1.73,6.81,3.69,3.03,0.48,...,0.71,0.00,0.42,10.22,2.94,1.83,4.10,0.90,1.31,2.40,6.76,0.82,13.40,1.24,8.35,3.85,4.13,0.76,0.14,0.65,0.20,0.44,0.39,0.0,0.26,0.04,0.08,0.0,14.28,6.91,4.38,0.09,0.15,0.05,0.0,1.07,0.10,0.64,0.37,0.54
4,1995,1,0,0,0,0,0,446.0,99.00,74.23,16.88,55.35,13.52,29.82,71.75,39.24,4.26,0.67,0.45,0.00,0.00,0.00,0.22,3.59,13.45,15.25,4.04,0.90,1.35,0.00,6.73,1.57,0.00,0.00,4.48,0.90,6.73,4.04,2.47,0.00,...,0.00,0.00,0.00,9.19,1.35,0.90,4.71,0.45,2.24,2.69,2.47,1.35,12.56,0.90,6.95,4.48,6.28,0.00,0.22,0.22,0.00,1.35,0.45,0.0,0.45,0.00,0.00,0.0,133.41,128.48,1.79,0.67,0.00,0.00,0.0,0.67,0.00,0.22,1.35,0.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074,2017,1,0,0,0,0,1,15470.0,97.88,70.65,19.22,85.54,27.19,29.61,75.84,46.05,7.32,3.41,0.92,1.54,0.12,0.38,0.45,3.92,10.34,17.31,4.82,2.46,5.00,0.54,8.13,4.09,1.71,0.72,4.43,1.47,5.22,4.27,0.85,0.17,...,0.18,0.01,0.08,8.84,2.82,2.34,2.59,1.16,0.95,1.71,5.69,1.00,12.00,1.41,6.93,3.76,5.27,0.50,0.10,0.46,0.15,0.05,0.40,0.0,0.23,0.06,0.12,0.0,17.43,7.33,4.45,0.08,0.06,0.01,0.0,1.46,0.12,0.48,2.39,1.05
5075,2017,1,0,0,0,0,1,24988.0,97.96,74.86,19.68,68.73,24.62,33.85,77.60,45.02,7.01,3.43,0.73,1.85,0.03,0.35,0.46,3.58,10.00,17.09,4.26,2.04,5.66,0.42,7.58,4.35,1.93,0.62,2.35,1.46,7.88,5.05,2.77,0.88,...,0.30,0.00,0.14,12.04,3.68,2.68,4.47,0.92,1.27,1.45,5.48,0.94,13.84,1.26,9.16,3.43,5.37,0.39,0.19,0.44,0.29,0.22,0.39,0.0,0.19,0.04,0.16,0.0,14.39,6.38,4.37,0.05,0.06,0.04,0.0,1.24,0.04,0.69,0.76,0.76
5076,2002,0,0,0,0,0,0,178.0,99.00,86.95,14.33,94.47,35.60,36.52,69.10,35.96,0.56,0.00,0.00,0.00,0.00,0.00,0.00,0.56,14.61,18.54,1.12,0.00,1.12,0.00,4.49,4.49,0.00,0.00,7.87,0.00,4.49,4.49,0.00,0.00,...,0.00,0.00,0.00,8.43,2.25,0.00,3.37,0.00,2.81,2.81,0.56,0.00,14.04,0.56,10.67,2.81,7.87,0.00,0

In [ ]:
#Download first file which is just motion queries, policy, and conflict
dfSC.drop(['meetings'], axis = 1, inplace= True)
dfSC.to_csv('SC_Query_Clean')

In [ ]:
#Download second file which includes everything

dfSC1.to_csv('SC_Query_LIWC')